In [9]:
import pandas as pd
from sklearn.metrics import confusion_matrix

MODEL_NAME = "PtLsOIE"


def get_dev_result(model_name: str):
    model_path = "..\evaluations\\" + model_name + "_dev.txt"
    with open(model_path, "r", encoding="utf-8") as f:
        lines = f.readlines()
        lines = [line.strip().split(" ") for line in lines]
        y_true = [line[1] for line in lines if len(line) == 3]
        y_pred = [line[2] for line in lines if len(line) == 3]
        return y_true, y_pred
y_true, y_pred = get_dev_result(MODEL_NAME)

def get_confusion_matrix(y_true, y_pred):
    labels = list(set(y_true))
    df = pd.DataFrame(
        data=confusion_matrix(y_true, y_pred, labels=labels),
        columns=labels,
        index=labels,
    )
    df.index.name = "Y-True"
    df.columns.name = "Y-Pred"
    return df

#
# Local (metrics per class)
#
df = get_confusion_matrix(y_true, y_pred)

labels = list(set(y_true))
tps = {}
fps = {}
fns = {}
for label in labels:
    tps[label] = df.loc[label, label]
    fps[label] = df[label].sum() - tps[label]
    fns[label] = df.loc[label].sum() - tps[label]

#
# Global
#
micro_averages = {}
macro_averages = {}

correct_predictions = sum(tps.values())

total_predictions = df.values.sum()
accuracy_global = round(correct_predictions / total_predictions,4 ) if total_predictions > 0. else 0.

print("#-- Local measures --#")
print("True Positives:", tps)
print("False Positives:", fps)
print("False Negatives:", fns)

print("\n#-- Global measures --#")
print("Correct predictions:", correct_predictions)
print("Total predictions:", total_predictions)
print("Accuracy:", accuracy_global)

#-- Local measures --#
True Positives: {'B-V': 29, 'O': 642, 'I-ARG1': 5, 'B-ARG1': 5, 'I-ARG0': 34, 'I-V': 0, 'B-ARG0': 30}
False Positives: {'B-V': 15, 'O': 244, 'I-ARG1': 1, 'B-ARG1': 3, 'I-ARG0': 5, 'I-V': 0, 'B-ARG0': 16}
False Negatives: {'B-V': 22, 'O': 25, 'I-ARG1': 51, 'B-ARG1': 46, 'I-ARG0': 50, 'I-V': 69, 'B-ARG0': 21}

#-- Global measures --#
Correct predictions: 745
Total predictions: 1029
Accuracy: 0.724


In [11]:
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred, digits=4))

              precision    recall  f1-score   support

      B-ARG0     0.6522    0.5882    0.6186        51
      B-ARG1     0.6250    0.0980    0.1695        51
         B-V     0.6591    0.5686    0.6105        51
      I-ARG0     0.8718    0.4048    0.5528        84
      I-ARG1     0.8333    0.0893    0.1613        56
         I-V     0.0000    0.0000    0.0000        69
           O     0.7246    0.9625    0.8268       667

    accuracy                         0.7240      1029
   macro avg     0.6237    0.3874    0.4199      1029
weighted avg     0.6822    0.7240    0.6592      1029



E:\poetry\venvs\flair-oie-_ETNQf08-py3.9\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
E:\poetry\venvs\flair-oie-_ETNQf08-py3.9\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
E:\poetry\venvs\flair-oie-_ETNQf08-py3.9\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
df.style.background_gradient(cmap="Purples")

Y-Pred,B-V,O,I-ARG1,B-ARG1,I-ARG0,I-V,B-ARG0
Y-True,,,,,,,
B-V,29,22,0,0,0,0,0
O,9,642,0,1,5,0,10
I-ARG1,0,51,5,0,0,0,0
B-ARG1,0,44,0,5,0,0,2
I-ARG0,0,46,0,0,34,0,4
I-V,6,60,1,2,0,0,0
B-ARG0,0,21,0,0,0,0,30


In [40]:
from transformers import MarianMTModel, MarianTokenizer, pipeline

pipe = pipeline("translation", model="Helsinki-NLP/opus-mt-tc-big-en-pt")
sents = ["The world is very big", "I am a student", "I am a teacher", "I live in Brazil"]
model_name = "Helsinki-NLP/opus-mt-tc-big-en-pt"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)
sent = "The Olympic Games take place every four years."
print(pipe(sent))

E:\poetry\venvs\flair-oie-_ETNQf08-py3.9\lib\site-packages\transformers\models\marian\tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


[{'translation_text': 'Os Jogos Olímpicos acontecem a cada quatro anos.'}]
